In [9]:
from bs4 import BeautifulSoup
import json
import urllib3
import os.path

In [10]:
url = 'https://nothai.com/menu/'

In [11]:
http = urllib3.PoolManager()
response = http.request("GET", url)
html_doc = response.data.decode('utf-8')

In [12]:
soup = BeautifulSoup(html_doc)
menu = soup.find_all("section", {"class": "menu-main-parent-row"})

In [13]:
menu

[<section class="l-section wpb_row us_custom_6e74c4b0 menu-main-parent-row height_small"><div class="l-section-h i-cf"><div class="g-cols vc_row via_flex valign_top type_default stacking_default"><div class="vc_col-sm-12 wpb_column vc_column_container"><div class="vc_column-inner"><div class="wpb_wrapper"><h2 class="w-text us_custom_f87f6ed2"><span class="w-text-h"><span class="w-text-value">Select your Entree</span></span></h2><h3 class="w-text us_custom_0dd32b33"><span class="w-text-h"><span class="w-text-value">Noodles</span></span></h3><div class="g-cols wpb_row via_flex valign_top type_default stacking_default"><div class="vc_col-sm-12 wpb_column vc_column_container"><div class="vc_column-inner"><div class="wpb_wrapper"><div class="w-hwrapper align_justify valign_middle" style="--hwrapper-gap:0.2rem"><div class="wpb_text_column"><div class="wpb_wrapper"><p>1. Pad Thai!*</p>
 </div></div><div class="w-image align_none"><div class="w-image-h"><img alt="gluten free" class="attachment

In [14]:
class Food():
    def __init__(self, name, price, description, food_type):
        self.name = name
        self.price = price
        self.description = description
        self.food_type = food_type
        
    def __str__(self):
        s = f"{self.name}\n"
        s += f"price: {self.price}\n"
        s += f"description: {self.description}\n"
        return s

        
class TreeNode():
    def __init__(self, food_type, food, children):
        self.food_type = food_type
        self.food = food
        self.children = children
        
    def is_leaf(self):
        return len(self.children) == 0

In [15]:
def get_food_types(fname=None, menu=None):
    food_types = []
    if not fname:
        for i in range(len(menu)):
            if i == 0:
                food_types.append(menu[i].find_all("span")[2].text)
            else:
                food_types.append(menu[i].find("span").text)
        with open('food_types.txt', 'w') as f:
            for food in food_types:
                f.write(f"{food}\n")
    else:
        with open(fname, 'r') as f:
            for line in f:
                food_types.append(line.strip())
    return food_types

def get_food(food_type, fname=None, raw_list=None):
    food_list = []
    if not fname:
        i = 0
        name = ''
        price = ''
        description = ''
        flag = 0
        while i < len(raw_list):
            if i % 3 == 0:
                name = raw_list[i].text
                num = int(name.split('.')[0])
                # there are two "19" in the official menu webpage, fix that
                if flag == 1:
                    num += 1
                    name = str(num) + '.' + name.split('.')[1]
                if num == 19:
                    flag = 1
            elif i % 3 == 1:
                price = raw_list[i].text
            else:
                description = raw_list[i].text
                food_list.append(Food(name, price, description, food_type))
            i += 1
        fname = food_type + '.txt'
        with open(fname, 'w') as f:
            for food in food_list:
                f.write(f"{food.name}\n")
                f.write(f"{food.price}\n")
                f.write(f"{food.description}\n")
    else:
        i = 0
        name = ''
        price = ''
        description = ''
        with open(fname, 'r') as f:
            for line in f:
                if i % 3 == 0:
                    name = line.strip()
                elif i % 3 == 1:
                    price = line.strip()
                else:
                    description = line.strip()
                    food_list.append(Food(name, price, description, food_type))
                i += 1
    return food_list

def get_drinks(fname=None, raw_list=None):
    food_type = 'Drinks'
    food_list = []
    if not fname:
        i = 0
        name = ''
        price = ''
        description = ''
        while i < len(raw_list):
            if i % 2 == 0:
                name = raw_list[i].text
            else:
                if len(raw_list[i].text) > 1:
                    # Finish finding drinks
                    break
                price = raw_list[i].text
                food_list.append(Food(name, price, description, food_type))
            i += 1
        fname = food_type + '.txt'
        with open(fname, 'w') as f:
            for food in food_list:
                f.write(f"{food.name}\n")
                f.write(f"{food.price}\n")
    else:
        i = 0
        name = ''
        price = ''
        description = ''
        with open(fname, 'r') as f:
            for line in f:
                if i % 2 == 0:
                    name = line.strip()
                else:
                    price = line.strip()
                    food_list.append(Food(name, price, description, food_type))
                i += 1
    return food_list
                

def get_all_food(food_types, menu=None):
    food_map = {}
    i = 0
    for food_type in food_types:
        fname = food_type + '.txt'
        if food_type != 'Drinks':
            if os.path.isfile(fname):
                food_list = get_food(food_type, fname, None)
            else:
                raw_list = menu[i].find("div", {"class": "wpb_wrapper"}).find_all("p")
                food_list = get_food(food_type, None, raw_list)
            food_map[food_type] = food_list
        else:
            if os.path.isfile(fname):
                food_list = get_drinks(fname, None)
            else:
                raw_list = menu[i].find("div", {"class": "wpb_wrapper"}).find_all("p")
                food_list = get_drinks(None, raw_list)
            food_map[food_type] = food_list
        i += 1
    return food_map

def build_tree(food_types, food_map):
    root = TreeNode('Entree', None, [])
    for food_type in food_types:
        node = TreeNode(food_type, None, [])
        for food in food_map[food_type]:
            leaf = TreeNode(food.name, food, None)
            node.children.append(leaf)
        root.children.append(node)
    return root

In [17]:
food_types = get_food_types('food_types.txt', None)
food_map = get_all_food(food_types, menu)

In [18]:
root = build_tree(food_types, food_map)

In [19]:
for food in food_map['Sides']:
    print(food)

18. Veggie Spring rolls (2)
price: 4
description: Shredded veggies fried in a wonton wrapper, with homemade plum sauce

19. Nogi Rolls (2)
price: 5
description: An Asian cheesesteak eggroll, with homemade plum sauce (Ann Arbor only)

20. Satay (4)
price: 6
description: Marinated chicken tenders on a bamboo stick w/ homemade peanut sauce (East Lansing only)

21. Crab Wontons (5)
price: 5
description: Cream cheese, crabmeat, scallions in fried wonton skin, with homemade plum sauce



In [42]:
url = 'https://nothai.com/about/'
http = urllib3.PoolManager()
response = http.request("GET", url)
html_doc = response.data.decode('utf-8')
soup = BeautifulSoup(html_doc, features="lxml")

In [51]:
about = soup.find_all("p", {'style': False, 'id': False})
about

[<p>This epic story begins on a cold winter day in February 2005. On that fateful day, four close friends united to form a pact to bring a modern Thai restaurant to the community. So the fellowship began, each person would bring to the table their own unique set of abilities. Ultimately, one partner, Noerung “No” Hang, rose above to become the head chef, and so the restaurant came to be known as “No Thai!”.</p>,
 <p>This name would come to bring amusement, confusion, and intrigue to the masses, but now you know No Thai!, and knowing is half the battle!</p>]

In [50]:
text = about[0].text
text = text.replace(u'\xa0', u' ')
text

'This epic story begins on a cold winter day in February 2005. On that fateful day, four close friends united to form a pact to bring a modern Thai restaurant to the community. So the fellowship began, each person would bring to the table their own unique set of abilities. Ultimately, one partner, Noerung “No” Hang, rose above to become the head chef, and so the restaurant came to be known as “No Thai!”.'

In [52]:
url = 'https://nothai.com/locations/'
http = urllib3.PoolManager()
response = http.request("GET", url)
html_doc = response.data.decode('utf-8')
soup = BeautifulSoup(html_doc, features="lxml")

In [71]:
locations = soup.find_all("section", {'class': 'wpb_row'})
locations[7].find("p").text.split("\n")[0]

'1952 S. Industrial Hwy, Ste F, Ann Arbor MI 48104'